## Setup

First, let's import the necessary NeqSim classes.

In [ ]:
import jpype
import jpype.imports
from jpype.types import *

# Start JVM if not already running
if not jpype.isJVMStarted():
    jpype.startJVM(classpath=['../target/classes'])

# Import NeqSim classes
from neqsim.thermo.system import SystemSrkEos, SystemPrEos, SystemSrkCPAstatoil
from neqsim.thermodynamicoperations import ThermodynamicOperations
from neqsim.thermo.util.derivatives import DifferentiableFlash, PropertyGradient, FlashGradients, FugacityJacobian

import numpy as np
import matplotlib.pyplot as plt

print("NeqSim derivatives package loaded successfully!")

## 1. Basic Property Gradients

Let's start with a simple example: computing gradients of mixture density with respect to temperature, pressure, and composition.

In [ ]:
# Create a simple binary mixture
system = SystemSrkEos(300.0, 50.0)  # 300 K, 50 bar
system.addComponent("methane", 1.0)
system.addComponent("propane", 0.3)
system.setMixingRule("classic")

# Run flash calculation
ops = ThermodynamicOperations(system)
ops.TPflash()
system.initProperties()

print(f"System at T = {system.getTemperature():.1f} K, P = {system.getPressure():.1f} bar")
print(f"Number of phases: {system.getNumberOfPhases()}")
print(f"Vapor fraction (β): {system.getBeta():.4f}")
print(f"Mixture density: {system.getDensity('kg/m3'):.2f} kg/m³")

In [ ]:
# Create DifferentiableFlash object
diff_flash = DifferentiableFlash(system)

# Compute density gradient
density_grad = diff_flash.computePropertyGradient("density")

print("=" * 60)
print("DENSITY GRADIENTS")
print("=" * 60)
print(f"Property: {density_grad.getPropertyName()} [{density_grad.getUnit()}]")
print(f"Current value: {density_grad.getValue():.4f}")
print(f"")
print(f"∂ρ/∂T = {density_grad.getDerivativeWrtTemperature():.6f} kg/m³/K")
print(f"∂ρ/∂P = {density_grad.getDerivativeWrtPressure():.6f} kg/m³/bar")
print(f"")
print("Composition derivatives (∂ρ/∂z_i):")
comp_derivs = density_grad.getDerivativeWrtComposition()
for i, dz in enumerate(comp_derivs):
    print(f"  {system.getComponent(i).getComponentName()}: {dz:.4f}")

## 2. Flash Gradients (K-values and Phase Fractions)

The `FlashGradients` class provides sensitivities of flash calculation results using the **implicit function theorem**.

At equilibrium, the residual equations F(y; θ) = 0 are satisfied. The implicit function theorem gives:

$$\frac{dy}{d\theta} = -\left(\frac{\partial F}{\partial y}\right)^{-1} \cdot \frac{\partial F}{\partial \theta}$$

where y = (K₁, ..., Kₙ, β) and θ = (T, P, z).

In [ ]:
# Compute flash gradients
flash_grads = diff_flash.computeFlashGradients()

print("=" * 60)
print("FLASH GRADIENTS")
print("=" * 60)

if flash_grads.isValid():
    print(f"Vapor fraction β = {flash_grads.getBeta():.4f}")
    print(f"")
    
    # K-value derivatives
    print("K-value sensitivities:")
    print("-" * 40)
    k_values = flash_grads.getKValues()
    dk_dT = flash_grads.getDKdT()
    dk_dP = flash_grads.getDKdP()
    
    for i in range(system.getNumberOfComponents()):
        name = system.getComponent(i).getComponentName()
        print(f"{name}:")
        print(f"  K = {k_values[i]:.4f}")
        print(f"  ∂K/∂T = {dk_dT[i]:.6f} 1/K")
        print(f"  ∂K/∂P = {dk_dP[i]:.6f} 1/bar")
    
    print(f"")
    print("Vapor fraction sensitivities:")
    print("-" * 40)
    print(f"∂β/∂T = {flash_grads.getDBetadT():.6f} 1/K")
    print(f"∂β/∂P = {flash_grads.getDBetadP():.6f} 1/bar")
else:
    print(f"Gradient computation failed: {flash_grads.getErrorMessage()}")

## 3. Fugacity Jacobians

The `FugacityJacobian` class contains derivatives of log fugacity coefficients, which are fundamental for flash calculations and stability analysis.

For each phase, we compute:
- ∂(ln φᵢ)/∂T
- ∂(ln φᵢ)/∂P  
- ∂(ln φᵢ)/∂nⱼ (the Jacobian matrix)

In [ ]:
# Extract fugacity Jacobians for each phase
print("=" * 60)
print("FUGACITY JACOBIANS")
print("=" * 60)

for phase_idx in range(system.getNumberOfPhases()):
    fug_jac = diff_flash.extractFugacityJacobian(phase_idx)
    
    print(f"\nPhase {phase_idx}: {fug_jac.getPhaseType()}")
    print("-" * 40)
    
    nc = fug_jac.getNumberOfComponents()
    ln_phi = fug_jac.getLnPhi()
    dln_phi_dT = fug_jac.getDlnPhidT()
    dln_phi_dP = fug_jac.getDlnPhidP()
    
    for i in range(nc):
        name = system.getComponent(i).getComponentName()
        print(f"  {name}:")
        print(f"    ln(φ) = {ln_phi[i]:.4f}")
        print(f"    ∂ln(φ)/∂T = {dln_phi_dT[i]:.6f} 1/K")
        print(f"    ∂ln(φ)/∂P = {dln_phi_dP[i]:.6f} 1/bar")

## 4. Validating Gradients with Finite Differences

Let's verify our analytical gradients by comparing them to numerical finite difference approximations.

In [ ]:
def compute_density_numerical(T, P, z_methane, z_propane):
    """Compute density at given conditions."""
    sys = SystemSrkEos(T, P)
    sys.addComponent("methane", z_methane)
    sys.addComponent("propane", z_propane)
    sys.setMixingRule("classic")
    ops = ThermodynamicOperations(sys)
    ops.TPflash()
    sys.initProperties()
    return sys.getDensity("kg/m3")

# Base conditions
T0, P0 = 300.0, 50.0
z1, z2 = 1.0, 0.3

# Compute numerical derivatives
eps = 1e-4
rho_base = compute_density_numerical(T0, P0, z1, z2)

# ∂ρ/∂T (numerical)
drho_dT_num = (compute_density_numerical(T0 + eps, P0, z1, z2) - 
               compute_density_numerical(T0 - eps, P0, z1, z2)) / (2 * eps)

# ∂ρ/∂P (numerical)
drho_dP_num = (compute_density_numerical(T0, P0 + eps, z1, z2) - 
               compute_density_numerical(T0, P0 - eps, z1, z2)) / (2 * eps)

# Compare with analytical
print("=" * 60)
print("GRADIENT VALIDATION (Analytical vs Numerical)")
print("=" * 60)
print(f"")
print(f"∂ρ/∂T:")
print(f"  Analytical: {density_grad.getDerivativeWrtTemperature():.6f} kg/m³/K")
print(f"  Numerical:  {drho_dT_num:.6f} kg/m³/K")
print(f"  Relative error: {abs(density_grad.getDerivativeWrtTemperature() - drho_dT_num) / abs(drho_dT_num) * 100:.2f}%")
print(f"")
print(f"∂ρ/∂P:")
print(f"  Analytical: {density_grad.getDerivativeWrtPressure():.6f} kg/m³/bar")
print(f"  Numerical:  {drho_dP_num:.6f} kg/m³/bar")
print(f"  Relative error: {abs(density_grad.getDerivativeWrtPressure() - drho_dP_num) / abs(drho_dP_num) * 100:.2f}%")

## 5. Multiple Property Gradients

The `DifferentiableFlash` class supports gradients for several properties:
- density
- enthalpy
- entropy
- Cp (heat capacity)
- compressibility (Z-factor)

In [ ]:
# Compute gradients for multiple properties
properties = ["density", "enthalpy", "entropy", "Cp", "compressibility"]

print("=" * 70)
print("SUMMARY OF ALL PROPERTY GRADIENTS")
print("=" * 70)
print(f"{'Property':<18} {'Value':<15} {'∂/∂T':<18} {'∂/∂P':<18}")
print("-" * 70)

for prop_name in properties:
    try:
        grad = diff_flash.computePropertyGradient(prop_name)
        print(f"{grad.getPropertyName():<18} {grad.getValue():<15.4f} {grad.getDerivativeWrtTemperature():<18.6f} {grad.getDerivativeWrtPressure():<18.6f}")
    except Exception as e:
        print(f"{prop_name:<18} Error: {str(e)[:40]}")

## 6. Sensitivity Analysis: Effect of Temperature on Phase Split

Let's use the gradients to perform a sensitivity analysis, showing how the vapor fraction changes with temperature.

In [ ]:
# Temperature sweep with gradient tracking
temperatures = np.linspace(250, 350, 50)
beta_values = []
beta_gradients = []
densities = []
density_gradients = []

for T in temperatures:
    sys = SystemSrkEos(float(T), 50.0)
    sys.addComponent("methane", 1.0)
    sys.addComponent("propane", 0.3)
    sys.setMixingRule("classic")
    
    ops = ThermodynamicOperations(sys)
    ops.TPflash()
    sys.initProperties()
    
    beta_values.append(sys.getBeta())
    densities.append(sys.getDensity("kg/m3"))
    
    # Compute gradients
    df = DifferentiableFlash(sys)
    flash_grads = df.computeFlashGradients()
    density_grad = df.computePropertyGradient("density")
    
    if flash_grads.isValid():
        beta_gradients.append(flash_grads.getDBetadT())
    else:
        beta_gradients.append(0.0)
    
    density_gradients.append(density_grad.getDerivativeWrtTemperature())

# Convert to numpy arrays
beta_values = np.array(beta_values)
beta_gradients = np.array(beta_gradients)
densities = np.array(densities)
density_gradients = np.array(density_gradients)

In [ ]:
# Plot results
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Vapor fraction vs Temperature
axes[0, 0].plot(temperatures, beta_values, 'b-', linewidth=2)
axes[0, 0].set_xlabel('Temperature (K)')
axes[0, 0].set_ylabel('Vapor Fraction β')
axes[0, 0].set_title('Vapor Fraction vs Temperature')
axes[0, 0].grid(True, alpha=0.3)

# dβ/dT vs Temperature
axes[0, 1].plot(temperatures, beta_gradients, 'r-', linewidth=2)
axes[0, 1].set_xlabel('Temperature (K)')
axes[0, 1].set_ylabel('∂β/∂T (1/K)')
axes[0, 1].set_title('Vapor Fraction Temperature Sensitivity')
axes[0, 1].grid(True, alpha=0.3)

# Density vs Temperature
axes[1, 0].plot(temperatures, densities, 'g-', linewidth=2)
axes[1, 0].set_xlabel('Temperature (K)')
axes[1, 0].set_ylabel('Density (kg/m³)')
axes[1, 0].set_title('Mixture Density vs Temperature')
axes[1, 0].grid(True, alpha=0.3)

# dρ/dT vs Temperature
axes[1, 1].plot(temperatures, density_gradients, 'm-', linewidth=2)
axes[1, 1].set_xlabel('Temperature (K)')
axes[1, 1].set_ylabel('∂ρ/∂T (kg/m³/K)')
axes[1, 1].set_title('Density Temperature Sensitivity')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Methane-Propane System at 50 bar', y=1.02, fontsize=14)
plt.show()

## 7. Multi-Component System Example

Let's work with a more complex natural gas mixture.

In [ ]:
# Create a natural gas mixture
ng_system = SystemSrkEos(280.0, 80.0)  # 280 K, 80 bar
ng_system.addComponent("nitrogen", 0.02)
ng_system.addComponent("CO2", 0.03)
ng_system.addComponent("methane", 0.85)
ng_system.addComponent("ethane", 0.05)
ng_system.addComponent("propane", 0.03)
ng_system.addComponent("n-butane", 0.02)
ng_system.setMixingRule("classic")

# Flash calculation
ng_ops = ThermodynamicOperations(ng_system)
ng_ops.TPflash()
ng_system.initProperties()

print("Natural Gas Mixture at T = 280 K, P = 80 bar")
print("=" * 60)
print(f"Number of phases: {ng_system.getNumberOfPhases()}")
print(f"Vapor fraction: {ng_system.getBeta():.4f}")
print(f"Mixture density: {ng_system.getDensity('kg/m3'):.2f} kg/m³")

In [ ]:
# Compute gradients for natural gas
ng_diff = DifferentiableFlash(ng_system)
ng_flash_grads = ng_diff.computeFlashGradients()

if ng_flash_grads.isValid():
    print("\nK-value Sensitivities for Natural Gas Components:")
    print("=" * 70)
    print(f"{'Component':<12} {'K-value':<12} {'∂K/∂T (1/K)':<18} {'∂K/∂P (1/bar)':<18}")
    print("-" * 70)
    
    k_vals = ng_flash_grads.getKValues()
    dk_dT = ng_flash_grads.getDKdT()
    dk_dP = ng_flash_grads.getDKdP()
    
    for i in range(ng_system.getNumberOfComponents()):
        name = ng_system.getComponent(i).getComponentName()
        print(f"{name:<12} {k_vals[i]:<12.4f} {dk_dT[i]:<18.6f} {dk_dP[i]:<18.6f}")
    
    print(f"\nVapor Fraction Sensitivities:")
    print(f"  ∂β/∂T = {ng_flash_grads.getDBetadT():.6f} 1/K")
    print(f"  ∂β/∂P = {ng_flash_grads.getDBetadP():.6f} 1/bar")
else:
    print(f"Gradient computation failed: {ng_flash_grads.getErrorMessage()}")

## 8. Composition Sensitivity Analysis

Let's analyze how the K-values depend on the feed composition (∂K/∂z).

In [ ]:
if ng_flash_grads.isValid():
    print("\nK-value Composition Sensitivities (∂Kᵢ/∂zⱼ):")
    print("=" * 90)
    
    # Get composition derivatives
    dk_dz = ng_flash_grads.getDKdz()
    comp_names = [ng_system.getComponent(i).getComponentName() for i in range(ng_system.getNumberOfComponents())]
    
    # Print header
    header = f"{'∂Kᵢ/∂zⱼ':<12}"
    for name in comp_names:
        header += f"{name[:8]:<10}"
    print(header)
    print("-" * 90)
    
    # Print matrix
    for i, name_i in enumerate(comp_names):
        row = f"{name_i:<12}"
        for j in range(len(comp_names)):
            row += f"{dk_dz[i][j]:<10.4f}"
        print(row)

## 9. Integration with Gradient-Based Optimization

The gradients can be used for optimization. Here's an example of finding the temperature that maximizes vapor fraction.

In [ ]:
def vapor_fraction_with_gradient(T, P=50.0):
    """Returns vapor fraction and its temperature gradient."""
    sys = SystemSrkEos(float(T), float(P))
    sys.addComponent("methane", 1.0)
    sys.addComponent("propane", 0.3)
    sys.setMixingRule("classic")
    
    ops = ThermodynamicOperations(sys)
    ops.TPflash()
    sys.initProperties()
    
    beta = sys.getBeta()
    
    df = DifferentiableFlash(sys)
    grads = df.computeFlashGradients()
    
    dbeta_dT = grads.getDBetadT() if grads.isValid() else 0.0
    
    return beta, dbeta_dT

# Gradient ascent to maximize vapor fraction
T = 280.0  # Starting temperature
learning_rate = 50.0
history = [(T, vapor_fraction_with_gradient(T)[0])]

print("Gradient Ascent to Maximize Vapor Fraction:")
print("=" * 50)

for iteration in range(20):
    beta, grad = vapor_fraction_with_gradient(T)
    
    # Update T using gradient
    T_new = T + learning_rate * grad
    T_new = max(200, min(400, T_new))  # Clamp to reasonable range
    
    if abs(T_new - T) < 0.01:
        print(f"Converged at iteration {iteration}")
        break
    
    T = T_new
    history.append((T, beta))
    
    if iteration % 5 == 0:
        print(f"Iteration {iteration}: T = {T:.2f} K, β = {beta:.4f}, ∂β/∂T = {grad:.6f}")

print(f"\nFinal: T = {T:.2f} K, β = {beta:.4f}")

## 10. Use Case: Machine Learning Integration

The gradients enable integration with ML frameworks. Here's a conceptual example showing how NeqSim gradients would flow through a neural network.

In [ ]:
# Conceptual example: Differentiable thermodynamics layer

class DifferentiableThermodynamicsLayer:
    """
    A conceptual layer that wraps NeqSim flash calculations
    and provides gradients for ML integration.
    
    In practice, this would be implemented as a custom autograd
    function in PyTorch or a custom_vjp in JAX.
    """
    
    def __init__(self, components, amounts):
        self.components = components
        self.amounts = amounts
    
    def forward(self, T, P):
        """Forward pass: compute density given T, P."""
        sys = SystemSrkEos(float(T), float(P))
        for comp, amt in zip(self.components, self.amounts):
            sys.addComponent(comp, amt)
        sys.setMixingRule("classic")
        
        ops = ThermodynamicOperations(sys)
        ops.TPflash()
        sys.initProperties()
        
        # Store for backward pass
        self._system = sys
        self._diff_flash = DifferentiableFlash(sys)
        
        return sys.getDensity("kg/m3")
    
    def backward(self, grad_output):
        """
        Backward pass: compute gradients w.r.t. T, P.
        
        grad_output: gradient of loss w.r.t. density (dL/dρ)
        Returns: (dL/dT, dL/dP) using chain rule
        """
        density_grad = self._diff_flash.computePropertyGradient("density")
        
        # Chain rule: dL/dT = dL/dρ * dρ/dT
        dL_dT = grad_output * density_grad.getDerivativeWrtTemperature()
        dL_dP = grad_output * density_grad.getDerivativeWrtPressure()
        
        return dL_dT, dL_dP

# Example usage
layer = DifferentiableThermodynamicsLayer(
    components=["methane", "propane"],
    amounts=[1.0, 0.3]
)

# Forward pass
rho = layer.forward(300.0, 50.0)
print(f"Forward pass: ρ = {rho:.4f} kg/m³")

# Backward pass (assuming dL/dρ = 1.0 for demonstration)
dL_dT, dL_dP = layer.backward(1.0)
print(f"Backward pass: dL/dT = {dL_dT:.6f}, dL/dP = {dL_dP:.6f}")

## Summary

The `neqsim.thermo.util.derivatives` package provides:

1. **PropertyGradient**: Container for ∂property/∂T, ∂property/∂P, ∂property/∂z
2. **FlashGradients**: Sensitivities of K-values and phase fractions via implicit differentiation
3. **FugacityJacobian**: Full Jacobian matrix of fugacity coefficients
4. **DifferentiableFlash**: Main interface combining all gradient computations

### Key Applications:
- **Process optimization**: Use gradients to optimize operating conditions
- **Sensitivity analysis**: Understand how outputs depend on inputs
- **ML integration**: Enable backpropagation through thermodynamic calculations
- **Uncertainty quantification**: Propagate uncertainties through flash calculations

### Mathematical Foundation:
The gradients are computed using the **implicit function theorem**, which provides exact derivatives without differentiating through iterative solvers. This approach is more accurate and efficient than finite differences.